##**4.3 - Data Preperation**

In [ ]:
import dask
from dask import dataframe as dd
from dask import array as da
from dask import bag as db
import pandas as pd
import io
import re
from google.colab import drive

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
drive.mount('/content/gdrive')

In [ ]:
dask_df = dd.read_csv('drive/MyDrive/DATA309/4.3_data.csv') #4.3_data includes all the data webscraped from amazon


In [ ]:

#from google.colab import files


#uploaded = files.upload()

In [ ]:
#filename = list(uploaded.keys())[0]

#with open(filename, 'wb') as f:
#    f.write(uploaded[filename])

#dask_df = dd.read_csv(filename)

#dask_df.head()

In [ ]:
print(dask_df.compute())

                                                   title   set_id  \
0                       !ATTENZIONE!! app non per tutti.    70418   
1      " It's amazing to watch them use the picture i...  6213583   
2      " It's amazing to watch them use the picture i...  6213583   
3                            "Lego"-Qualität in Bestform    10265   
4                                   #Love LEGO Star Wars    75357   
...                                                  ...      ...   
12127  🟨🟨🟨 SET DIVERTENTE, RICCO DI GIOCHI E AMBIENTA...    60372   
12128  🟨🟨🟨 SET DIVERTENTE, RICCO DI GIOCHI E AMBIENTA...    60372   
12129                        🦒 JIRAFA REINA POR UN DÍA 🦒    31150   
12130          🧩🧩 Quand Lego collabore avec Minecraft 🧩🧩    21255   
12131       🧱🧱 Teures Star Wars-Diorama vom Podrennen 🧱🧱    75380   

                                                    date  \
0                  Reviewed in Italy on 27 December 2019   
1         Reviewed in the United States on 16 April

FREQUENT ITEMS

In [ ]:
#useless words so we can ignore them:

import nltk
nltk.download('stopwords')

from collections import Counter
from nltk.corpus import stopwords


STOP_WORDS = set(stopwords.words('english'))
print(STOP_WORDS)


#nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


{'such', 'me', 'once', 'himself', 'not', 'out', 'our', 'through', 'has', "weren't", 'when', "needn't", 'ain', 'they', "you'll", 'this', "mustn't", 're', 'whom', 'why', "shouldn't", 'will', "wouldn't", 'his', "doesn't", 'you', 'their', 'nor', 'ourselves', 'an', 'more', "don't", "you'd", 'for', 'as', 'mightn', 'own', 'to', 'other', "isn't", 'him', 'needn', 'myself', 'with', 'which', 'where', 'y', 'that', 'being', 'over', 'below', 'hers', 'what', 'before', 'aren', 'off', 'shan', 'have', 'who', 'now', 'them', 'from', 'yours', "that'll", 'too', 'up', 'again', 'won', 'by', 'so', 'mustn', "it's", 'i', 'shouldn', 'same', 'itself', 'on', 'didn', 'do', 'or', 'doing', 'but', "didn't", 'herself', 'my', "mightn't", 'each', 'yourselves', 'isn', 'both', 'm', "shan't", 'wasn', 'there', 'further', 'because', 'don', 'weren', 'hadn', 'those', 'was', 'o', 'any', 'its', 'yourself', 'the', 'she', 'had', 'did', "you've", 'theirs', 'if', 'of', 'hasn', 'he', 'into', 'after', 'your', 'down', "hasn't", 'doesn', 

In [ ]:
dask_df = dask_df.dropna()
#data = data.dropna()

In [ ]:
!pip install translate
from translate import Translator
translator = Translator(to_lang="eng")

In [ ]:
#Testing the Translator

translation = translator.translate("こんにちは") #Hello in Japanese
print(translation)


#German
translation = translator.translate("Guten Morgen") #Good morning in german
print(f"Auto Translator from German: [{translation}]")

german_translator = Translator(to_lang="en", from_lang="de")
translation = german_translator.translate("Guten Morgen") #Good morning in german
print(f"Specified Translator from German: [{translation}]")


#ENLISH TO ENGLISH
translation = translator.translate("amazing")
print(f"Auto Translator from English: [{translation}]")

english_translator = Translator(to_lang="en", from_lang="en")
translation = english_translator.translate("amazing")
print(f"Specified translator from English: [{translation}]")



Hello
Auto Translator from German: [Tomorrow]
Specified Translator from German: [Good morning]
Auto Translator from English: [అద్భుతమైన]
Specified translator from English: [amazing]


Need to specify what language we are translating from using from_lang="xx" otherwise it does not get translated properly

In [ ]:
print(dask_df['date'].unique().compute())


0                   Reviewed in Italy on 27 December 2019
1          Reviewed in the United States on 16 April 2018
2       Reviewed in the United Kingdom on 9 November 2023
3                   Reviewed in Japan on 25 February 2024
4                  Reviewed in Germany on 7 November 2020
                              ...                        
4568                    Reviewed in Italy on 2 April 2024
4569                   Reviewed in Italy on 23 April 2022
4570                    Reviewed in Italy on 21 June 2024
4571                  Reviewed in Germany on 3 March 2024
4572                  Reviewed in Italy on 4 January 2023
Name: date, Length: 4573, dtype: string


In [ ]:
#Testing

country_to_lang = {
    "Australia": "en",
    "United States": "en",
    "United Kingdom": "en",
    "Germany": "de",
    "France": "fr",
    "Spain": "es",
    "Italy": "it",
    "Canada": "en",
    "Japan": "ja",
    "China": "zh",
    "Mexico": "es",
    "Poland": "pl",
    "Saudi Arabia": "ar",
    "Singapore": "si",
    "Sweden": "sv",
    "Netherlands": "nl",
    "Switzerland": "de",
    "Belgium": "nl",
    "Denmark": "da",
    "United Arab Emirates": "ar",
    "South Korea": "ko",
    "Turkey": "Tr"
}

# Function to extract the country from the date column
def extract_country(date_string):
    for country in country_to_lang.keys():
        if country in date_string:
            return country_to_lang[country]
    return "en"  # Default to English if country not found


print(extract_country('Reviewed in Turkey on 31 December 2023.'))
print(extract_country('Reviewed in Germany on 31 December 2023.'))
print(extract_country('Reviewed in France on 31 December 2023.'))
print(extract_country('Reviewed in Italy on 31 December 2023.'))
print(extract_country('Reviewed in Spain on 31 December 2023.'))


Tr
de
fr
it
es


We cannot extract the language code from the country reviewed in because there are many cases of users writing reviews in a language that is not a language from the country they reviewed in. E.g. A user writes a review in English but they are from the Netherlands.

- using NLTK find the most loved and most hated sets
  -  Use sentiment score from NLTK


https://www.nltk.org/howto/sentiment.html

In [ ]:
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentAnalyzer
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
#testing
sentences = ["VADER is smart, handsome, and funny.", # positive sentence example
    "VADER is smart, handsome, and funny!", # punctuation emphasis handled correctly (sentiment intensity adjusted)
    "VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
    "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
    "VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
    "VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!",# booster words & punctuation make this close to ceiling for score
    "The book was good.",         # positive sentence
    "The book was kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
    "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
    "A really bad, horrible book.",       # negative sentence with booster words
    "At least it isn't a horrible book.", # negated negative sentence with contraction
    ":) and :D",     # emoticons handled
    "",              # an empty string is correctly handled
    "Today sux",     #  negative slang handled
    "Today sux!",    #  negative slang with punctuation emphasis handled
    "Today SUX!",    #  negative slang with capitalization emphasis
   "Today kinda sux! But I'll get by, lol"] # mixed sentiment example with slang and constrastive conjunction "but""


paragraph = "It was one of the worst movies I've seen, despite good reviews. \
Unbelievably bad acting!! Poor direction. VERY poor production. The movie was bad. Very bad movie. VERY bad movie. VERY BAD movie. VERY BAD movie! "

In [ ]:
lines_list = tokenize.sent_tokenize(paragraph)
sentences.extend(lines_list)
print(lines_list)

["It was one of the worst movies I've seen, despite good reviews.", 'Unbelievably bad acting!!', 'Poor direction.', 'VERY poor production.', 'The movie was bad.', 'Very bad movie.', 'VERY bad movie.', 'VERY BAD movie.', 'VERY BAD movie!']


In [ ]:
for line in lines_list:
  sid = SentimentIntensityAnalyzer()
  ss = sid.polarity_scores(line)
  for k in sorted(ss):
    print('{0}: {1}, '.format(k, ss[k]), end='')
  print()

compound: -0.7584, neg: 0.394, neu: 0.606, pos: 0.0, 
compound: -0.6572, neg: 0.686, neu: 0.314, pos: 0.0, 
compound: -0.4767, neg: 0.756, neu: 0.244, pos: 0.0, 
compound: -0.6281, neg: 0.674, neu: 0.326, pos: 0.0, 
compound: -0.5423, neg: 0.538, neu: 0.462, pos: 0.0, 
compound: -0.5849, neg: 0.655, neu: 0.345, pos: 0.0, 
compound: -0.6732, neg: 0.694, neu: 0.306, pos: 0.0, 
compound: -0.7398, neg: 0.724, neu: 0.276, pos: 0.0, 
compound: -0.7616, neg: 0.735, neu: 0.265, pos: 0.0, 


In [ ]:
for line in sentences:
  sid = SentimentIntensityAnalyzer()
  ss = sid.polarity_scores(line)
  for k in sorted(ss):
    print('{0}: {1}, '.format(k, ss[k]), end='')
  print()

compound: 0.8316, neg: 0.0, neu: 0.254, pos: 0.746, 
compound: 0.8439, neg: 0.0, neu: 0.248, pos: 0.752, 
compound: 0.8545, neg: 0.0, neu: 0.299, pos: 0.701, 
compound: 0.9227, neg: 0.0, neu: 0.246, pos: 0.754, 
compound: 0.9342, neg: 0.0, neu: 0.233, pos: 0.767, 
compound: 0.9469, neg: 0.0, neu: 0.294, pos: 0.706, 
compound: 0.4404, neg: 0.0, neu: 0.508, pos: 0.492, 
compound: 0.3832, neg: 0.0, neu: 0.657, pos: 0.343, 
compound: -0.7042, neg: 0.327, neu: 0.579, pos: 0.094, 
compound: -0.8211, neg: 0.791, neu: 0.209, pos: 0.0, 
compound: 0.431, neg: 0.0, neu: 0.637, pos: 0.363, 
compound: 0.7925, neg: 0.0, neu: 0.124, pos: 0.876, 
compound: 0.0, neg: 0.0, neu: 0.0, pos: 0.0, 
compound: -0.3612, neg: 0.714, neu: 0.286, pos: 0.0, 
compound: -0.4199, neg: 0.736, neu: 0.264, pos: 0.0, 
compound: -0.5461, neg: 0.779, neu: 0.221, pos: 0.0, 
compound: 0.5249, neg: 0.138, neu: 0.517, pos: 0.344, 
compound: -0.7584, neg: 0.394, neu: 0.606, pos: 0.0, 
compound: -0.6572, neg: 0.686, neu: 0.314, p

In [ ]:
print(dask_df)

Dask DataFrame Structure:
                title set_id    date content rating num_pieces manufacturer_recommended age release_date date_first_available
npartitions=1                                                                                                                
               string  int64  string  string  int64      int64                       string       string               string
                  ...    ...     ...     ...    ...        ...                          ...          ...                  ...
Dask Name: dropna, 3 graph layers


In [ ]:
print(dask_df['content'].compute())

0        MOLTO DELUSO!! Ho acquistato diversi set della...
1        These Legos are a constant source of entertain...
2        These Legos are a constant source of entertain...
4        Not necessarily the most adult focused set, ho...
5        Not necessarily the most adult focused set, ho...
                               ...                        
12126    Herzenswunsch des Kindes zum 9. Geburtstag und...
12127    Ho trovato molto carino e ben fatto questo set...
12128    Ho trovato molto carino e ben fatto questo set...
12130    Lego Minecraft "L'Embuscade au Portail du Neth...
12131    Dieses mittelgroße Set aus dem Star Wars Unive...
Name: content, Length: 8663, dtype: string


Langdetect module can be used to solve the problem of users writing reviews in different languages from their country.

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=0f66ba60836fe906f678689686624453c7eb51cfb7c6bf043b65a43ec5ccbaa8
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from langdetect import detect

In [ ]:
#Testing:
print(detect("War doesn't show who's right, just who's left."))

print(detect("buenos dias y buenas noches"))

print(detect("Guten Morgen"))

print(detect("j'aime les lego"))

print(detect("bel prodotto"))

print(detect("惊人的产品"))

print(detect("nie jest to dobry produkt"))

print(detect("منتج سيء حقا")) #arabic ar

print(detect("riktigt bra produkt tack")) #swedish sv

print(detect("echt een leuk product, bedankt")) #dutch nl

en
es
de
fr
it
zh-cn
pl
ar
sv
nl


In [ ]:
print(dask_df.head())

                                               title   set_id  \
0                   !ATTENZIONE!! app non per tutti.    70418   
1  " It's amazing to watch them use the picture i...  6213583   
2  " It's amazing to watch them use the picture i...  6213583   
4                               #Love LEGO Star Wars    75357   
5                               #Love LEGO Star Wars    75357   

                                                date  \
0              Reviewed in Italy on 27 December 2019   
1     Reviewed in the United States on 16 April 2018   
2     Reviewed in the United States on 16 April 2018   
4  Reviewed in the United Kingdom on 9 November 2023   
5  Reviewed in the United Kingdom on 9 November 2023   

                                             content  rating  num_pieces  \
0  MOLTO DELUSO!! Ho acquistato diversi set della...       2         173   
1  These Legos are a constant source of entertain...       5         244   
2  These Legos are a constant source of ente

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

# Function to translate to English using the `translate` package
def translate_to_english(text):
    try:
        detected_lang = detect_language(text)
        if detected_lang != 'en':
            translated = translator.translate(text)
            return translated
        else:
            return text
    except Exception as e:
        return str(e)



In [ ]:
print(dask_df.head())

                                               title   set_id  \
0                   !ATTENZIONE!! app non per tutti.    70418   
1  " It's amazing to watch them use the picture i...  6213583   
2  " It's amazing to watch them use the picture i...  6213583   
4                               #Love LEGO Star Wars    75357   
5                               #Love LEGO Star Wars    75357   

                                                date  \
0              Reviewed in Italy on 27 December 2019   
1     Reviewed in the United States on 16 April 2018   
2     Reviewed in the United States on 16 April 2018   
4  Reviewed in the United Kingdom on 9 November 2023   
5  Reviewed in the United Kingdom on 9 November 2023   

                                             content  rating  num_pieces  \
0  MOLTO DELUSO!! Ho acquistato diversi set della...       2         173   
1  These Legos are a constant source of entertain...       5         244   
2  These Legos are a constant source of ente

In [ ]:
#make all reviews lowercase for more accurate translation:
dask_df['content'] = dask_df['content'].str.lower()

In [ ]:
print(dask_df.head())

                                               title   set_id  \
0                   !ATTENZIONE!! app non per tutti.    70418   
1  " It's amazing to watch them use the picture i...  6213583   
2  " It's amazing to watch them use the picture i...  6213583   
4                               #Love LEGO Star Wars    75357   
5                               #Love LEGO Star Wars    75357   

                                                date  \
0              Reviewed in Italy on 27 December 2019   
1     Reviewed in the United States on 16 April 2018   
2     Reviewed in the United States on 16 April 2018   
4  Reviewed in the United Kingdom on 9 November 2023   
5  Reviewed in the United Kingdom on 9 November 2023   

                                             content  rating  num_pieces  \
0  molto deluso!! ho acquistato diversi set della...       2         173   
1  these legos are a constant source of entertain...       5         244   
2  these legos are a constant source of ente

In [ ]:
dask_df['translated_content'] = dask_df['content'].apply(translate_to_english, meta=('content', 'str'))


#print(dask_df.head())

The Translator module has a 5000 character limit so we will use deep-translator and use Google Translate's services

In [ ]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

In [ ]:
from deep_translator import GoogleTranslator

In [ ]:
text = 'Mir gefällt dieses Lego-Set sehr, vielen Dank'
print(detect(text))
translated = GoogleTranslator(source=detect(text), target='en').translate(text)
print(translated)

de
I really like this Lego set, thank you very much


In [ ]:
def translate_to_english(text):
    try:
        detected_lang = detect_language(text)
        if detected_lang != 'en':
            translated = GoogleTranslator(source=detect(text), target='en').translate(text)
            return translated
        else:
            return text
    except Exception as e:
        return str(e)

In [ ]:
#create new translated column:

dask_df['translated_content'] = dask_df['content'].apply(translate_to_english, meta=('content', 'str'))

In [ ]:
print(dask_df.head())

In [ ]:
dask_df.to_csv('final_translated.csv')

['/content/final_translated.csv/0.part']

In [ ]:
data = dd.read_csv('drive/MyDrive/DATA309/fixed translate.csv')

In [ ]:
print(data.tail())

In [ ]:
data = data.drop_duplicates(subset=['translated_content'])


In [ ]:
print(data)

Dask DataFrame Structure:
              Unnamed: 0   title set_id    date content rating num_pieces manufacturer_recommended age release_date date_first_available translated_content
npartitions=1                                                                                                                                              
                   int64  string  int64  string  string  int64      int64                       string       string               string             string
                     ...     ...    ...     ...     ...    ...        ...                          ...          ...                  ...                ...
Dask Name: drop-duplicates-agg, 4 graph layers


In [ ]:
sid = SentimentIntensityAnalyzer()

results = []

for line in data['translated_content'].compute():
    ss = sid.polarity_scores(line)
    result = {
        'review': line,
        'compound': ss['compound'],
        'neg': ss['neg'],
        'neu': ss['neu'],
        'pos': ss['pos']
    }
    results.append(result)

sentiment_df = dd.from_pandas(pd.DataFrame(results), npartitions=1)

print(sentiment_df.compute())

                                                 review  compound    neg  \
0     very disappointed!! I bought several sets of t...   -0.8343  0.112   
1     these legos are a constant source of entertain...    0.9544  0.000   
2     not necessarily the most adult focused set, ho...    0.5914  0.000   
3     A Christmas present from a grandmother to her ...    0.9343  0.018   
4                                  No features in text.   -0.2960  0.423   
...                                                 ...       ...    ...   
5767  The product is very very good, small but reall...    0.9577  0.000   
5768  A child's heart's desire for his 9th birthday ...    0.9337  0.051   
5769  I found this Lego Police Training Academy set ...    0.9938  0.022   
5770  lego minecraft "the ambush at the nether porta...    0.8519  0.022   
5771  This medium-sized set from the Star Wars unive...    0.8066  0.059   

        neu    pos  
0     0.862  0.027  
1     0.680  0.320  
2     0.838  0.162  
3  

In [ ]:
#Merge results:
data = data.merge(sentiment_df, left_on='translated_content', right_on='review', how='left')

In [ ]:
#print(data.compute())

In [ ]:
data.to_csv('new.csv')

In [ ]:
#Now add the 'language' column

In [ ]:
data = data.assign(language=data['content'].apply(detect_language, meta=('content', 'str')))

In [ ]:
#remove rows with review character length less than 25 because the very short reviews get mis interpreted for a different language sometimes.
data = data[data['content'].str.len() >= 25]

In [ ]:
print(data.tail())

      Unnamed: 0                                              title  set_id  \
5767       12124                                         👍😘muhteşem   76265   
5768       12126                                                😎😎😎   71411   
5769       12127  🟨🟨🟨 SET DIVERTENTE, RICCO DI GIOCHI E AMBIENTA...   60372   
5770       12130          🧩🧩 Quand Lego collabore avec Minecraft 🧩🧩   21255   
5771       12131       🧱🧱 Teures Star Wars-Diorama vom Podrennen 🧱🧱   75380   

                                        date  \
5767    Reviewed in Turkey on 24 August 2024   
5768  Reviewed in Germany on 27 October 2023   
5769     Reviewed in Italy on 4 January 2023   
5770  Reviewed in France on 28 February 2024   
5771      Reviewed in Germany on 8 June 2024   

                                                content  rating  num_pieces  \
5767  ürün çok çok iyi ufak ama cidden çok güzel. ko...       5         357   
5768  herzenswunsch des kindes zum 9. geburtstag und...       5        2807   

In [ ]:
data.to_csv('sentiment analysis.csv')

['/content/sentiment analysis.csv/0.part']

## **4.31 - Frequent Items:**

In [ ]:
one_star = data[data['rating'] == 1]['translated_content'].compute()
two_star = data[data['rating'] == 2]['translated_content'].compute()
three_star = data[data['rating'] == 3]['translated_content'].compute()
four_star = data[data['rating'] == 4]['translated_content'].compute()
five_star = data[data['rating'] == 5]['translated_content'].compute()

# clean and tokenize reviews, removing stopwords
def tokenize_reviews(reviews, stop_words):
    # Combine all reviews into one large string
    all_reviews = ' '.join(reviews)
    # Remove special characters and split into words
    words = re.findall(r'\b\w+\b', all_reviews.lower())
    # Remove stopwords
    meaningful_words = [word for word in words if word not in stop_words]
    return meaningful_words

one_star_words = tokenize_reviews(one_star, STOP_WORDS)
two_star_words = tokenize_reviews(two_star, STOP_WORDS)
three_star_words = tokenize_reviews(three_star, STOP_WORDS)
four_star_words = tokenize_reviews(four_star, STOP_WORDS)
five_star_words = tokenize_reviews(five_star, STOP_WORDS)


def most_common_words(words, n=20):
    return Counter(words).most_common(n)

print("Most common words in 1 star reviews")
for word, count in most_common_words(one_star_words):
    print(f"{word}: {count}")

print("Most common words in 2 star reviews")
for word, count in most_common_words(two_star_words):
    print(f"{word}: {count}")

print("Most common words in 3 star reviews")
for word, count in most_common_words(three_star_words):
    print(f"{word}: {count}")

print("Most common words in 4 star reviews")
for word, count in most_common_words(four_star_words):
    print(f"{word}: {count}")

print("Most common words in 5 star reviews")
for word, count in most_common_words(five_star_words):
    print(f"{word}: {count}")

In [ ]:
#download results to txt file
def write_most_common_words_to_file(file_path, star_rating, words):
    with open(file_path, 'a') as file:
        file.write(f"\nMost common words in {star_rating} star reviews\n")
        for word, count in most_common_words(words):
            file.write(f"{word}: {count}\n")

output_file = "common_words_by_rating.txt"


with open(output_file, 'w') as file:
    file.write("Common Words in LEGO Reviews\n")


write_most_common_words_to_file(output_file, 1, one_star_words)
write_most_common_words_to_file(output_file, 2, two_star_words)
write_most_common_words_to_file(output_file, 3, three_star_words)
write_most_common_words_to_file(output_file, 4, four_star_words)
write_most_common_words_to_file(output_file, 5, five_star_words)

print(f"Most common words have been written to {output_file}")

##**4.32 - Frequent Itemsets**

In [ ]:
#Frequent Itemsets (Apriori)


stop_words = set(stopwords.words('english'))



one_star_reviews = data[data['rating'] == 1]['translated_content'].compute().tolist()
two_star_reviews = data[data['rating'] == 2]['translated_content'].compute().tolist()
three_star_reviews = data[data['rating'] == 3]['translated_content'].compute().tolist()
four_star_reviews = data[data['rating'] == 4]['translated_content'].compute().tolist()
five_star_reviews = data[data['rating'] == 5]['translated_content'].compute().tolist()


# Convert lists of reviews to Dask bags


one_star_bag = db.from_sequence(one_star_reviews)
two_star_bag = db.from_sequence(two_star_reviews)
three_star_bag = db.from_sequence(three_star_reviews)
four_star_bag = db.from_sequence(four_star_reviews)
five_star_bag = db.from_sequence(five_star_reviews)

def clean_and_tokenize(review, stop_words):
    words = re.findall(r'\b\w+\b', review.lower())
    meaningful_words = [word for word in words if word not in stop_words]
    return ' '.join(meaningful_words)



one_star_bag = one_star_bag.map(lambda translated_content: clean_and_tokenize(translated_content, stop_words))
two_star_bag = two_star_bag.map(lambda translated_content: clean_and_tokenize(translated_content, stop_words))
three_star_bag = three_star_bag.map(lambda translated_content: clean_and_tokenize(translated_content, stop_words))
four_star_bag = four_star_bag.map(lambda translated_content: clean_and_tokenize(translated_content, stop_words))
five_star_bag = five_star_bag.map(lambda translated_content: clean_and_tokenize(translated_content, stop_words))


# Apriori
def a_priori_step1(text_file_bag, support=100):
    item_counts = (
        text_file_bag
        .map(lambda line: line.split())
        .flatten()
        .frequencies()
        .filter(lambda x: x[1] >= support)  # Filter frequent items
        .map(lambda x: x[0])  # Extract item names
        .compute()
    )
    frequent_items = set(item_counts)  # Convert to set
    return frequent_items

def generate_pairs(line, frequent_items):
    items = set(line.split())  # Convert list to set to remove duplicates
    pairs = [(item1, item2) for item1 in items for item2 in items if item1 < item2]
    return [(pair[0], pair[1]) for pair in pairs if pair[0] in frequent_items and pair[1] in frequent_items]

def a_priori(text_file_bag, support=100):
    frequent_items = a_priori_step1(text_file_bag, support)
    frequent_items_broadcast = db.from_sequence([frequent_items]).persist()

    pairs_bag = text_file_bag.map(generate_pairs, frequent_items=frequent_items).flatten()

    # Reduce pairs into pair counts efficiently
    pair_counts = pairs_bag.frequencies()

    # Filter pairs based on the support threshold
    frequent_pairs = pair_counts.filter(lambda x: x[1] >= support).compute()

    return frequent_pairs

# Apply the Apriori algorithm to each star rating
one_star_pairs = a_priori(one_star_bag, support=5)
two_star_pairs = a_priori(two_star_bag, support=5) #lowered support for one and two star reviews because there arent many
three_star_pairs = a_priori(three_star_bag, support=10)
four_star_pairs = a_priori(four_star_bag, support=10)
five_star_pairs = a_priori(five_star_bag, support=10)

# Sort airs by frequency

sorted_one_pairs = sorted(one_star_pairs, key=lambda x: x[1], reverse=True)
sorted_two_pairs = sorted(two_star_pairs, key=lambda x: x[1], reverse=True)
sorted_three_pairs = sorted(three_star_pairs, key=lambda x: x[1], reverse=True)
sorted_four_pairs = sorted(four_star_pairs, key=lambda x: x[1], reverse=True)
sorted_five_pairs = sorted(five_star_pairs, key=lambda x: x[1], reverse=True)




In [ ]:
print(sorted_one_pairs)
print(sorted_two_pairs)
print(sorted_three_pairs)
print(sorted_four_pairs)
print(sorted_five_pairs)

In [ ]:
def write_apirori_to_file(file_path, star_rating, pairs):
    with open(file_path, 'a') as file:
        file.write(f"\nFrequent pairs in {star_rating} star reviews\n")
        for word, count in most_common_words(pairs):
            file.write(f"{word}\n")

output_file = "Apriori.txt"


with open(output_file, 'w') as file:
    file.write("Frequent Pairs in Lego Reviews:\n")


write_apirori_to_file(output_file, 1, sorted_one_pairs)
write_apirori_to_file(output_file, 2, sorted_two_pairs)
write_apirori_to_file(output_file, 3, sorted_three_pairs)
write_apirori_to_file(output_file, 4, sorted_four_pairs)
write_apirori_to_file(output_file, 5, sorted_five_pairs)

print(f"Most common words have been written to {output_file}")